# Homework

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [2]:
train = pd.read_parquet("yellow_tripdata_2023-01.parquet")
valid = pd.read_parquet("yellow_tripdata_2023-02.parquet")
print(f"There are {train.shape[1]} columns")

There are 19 columns


## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

In [3]:
train["duration"] = (train['tpep_dropoff_datetime'] - train["tpep_pickup_datetime"]).dt.total_seconds() / 60
valid["duration"] = (valid['tpep_dropoff_datetime'] - valid["tpep_pickup_datetime"]).dt.total_seconds() / 60
print(f"The standard deviation of the trips duration in January is {train['duration'].std()}")

The standard deviation of the trips duration in January is 42.594351241920904


## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [4]:
fraction = ((train["duration"] >=1) & (train["duration"] <= 60)).mean()
train = train[(train["duration"] >=1) & (train["duration"] <= 60)].copy()
valid = valid[(valid["duration"] >=1) & (valid["duration"] <= 60)].copy()
print(f"The fraction of records left after dropping the outliers is {round(fraction * 100)}")

The fraction of records left after dropping the outliers is 98


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [5]:
CATEGORICAL = ['PULocationID', 'DOLocationID']
train[CATEGORICAL] = train[CATEGORICAL].astype(str)
train_dict = train[CATEGORICAL].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dict)
print(f"The dimensionality of this matrix (number of columns) is {X_train.shape[1]}")

The dimensionality of this matrix (number of columns) is 515


## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters, where duration is the response variable
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [6]:
target = "duration"
y_train = train[target].values
lr = LinearRegression()
lr.fit(X_train, y_train)
train_pred = lr.predict(X_train)
train_rmse = root_mean_squared_error(y_train, train_pred)
print(f"RMSE on train is {round(train_rmse, 2)}")

RMSE on train is 7.65


## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

In [7]:
valid[CATEGORICAL] = valid[CATEGORICAL].astype(str)
valid_dict = valid[CATEGORICAL].to_dict(orient='records')
X_valid = dv.transform(valid_dict)
y_valid = valid[target].values
valid_pred = lr.predict(X_valid)
valid_rmse = root_mean_squared_error(y_valid, valid_pred)
print(f"RMSE on validation is {round(valid_rmse, 2)}")

RMSE on validation is 7.81


## Submit the results

* Submit your results here: https://courses.datatalks.club/mlops-zoomcamp-2024/homework/hw1
* If your answer doesn't match options exactly, select the closest one